In [1]:
# import dependencies
import os, inspect
import sqlalchemy as sqlalc
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base

In [2]:
# get path to current and root directories
CURR_DIR = os.path.dirname(inspect.getabsfile(inspect.currentframe()))
ROOT_DIR = os.path.dirname(CURR_DIR)\

I start off by creating a SQLAlchemy engine that allows us to connect and interface with sql database.

In [3]:
# create sqlite engine and connect to data base
fname = os.path.join(ROOT_DIR, "bbbioboard", "data", "ext", "bellybutton.sqlite")
engine = sqlalc.create_engine(f"sqlite:///{fname}")

Having the database connected to the engine, we will list up the available tables:

In [4]:
inspector = sqlalc.engine.reflection.Inspector.from_engine(engine)
inspector.get_table_names()

['sample_metadata', 'samples']

This show that the current database has two tables with names of 'sample_metadata' and 'samples. These table names allows us to dig into the tables data and explore the table metadata.

In [5]:
inspector.get_columns("samples")

[{'name': 'otu_id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'otu_label',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': '940',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': '941',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': '943',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': '944',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': '945',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': '946',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key'

It seems that there is something wrong with column names of the 'samples' table. Most of the column names here start with numbers. The Python parser forbids the variables with such naming and so making query will not be straightforwad. As a way around, we should consider to rename such columns with a prefix as we map classes to tables. To make this happen, we apply declrative_mapping. Delcrative mapping will allows us to have more control over the mapping process. We exploit declrative mapping to add prefix at the begining of column keys:

In [6]:
Base = declarative_base(engine)
class samplesTable(Base):
    __tablename__ = "samples"
    __table_args__ = {"autoload": True}
    __mapper_args__ = {'column_prefix':'bb_'}
    

class samplesMetaDataTable(Base):
    __tablename__ = "sample_metadata"
    __table_args__ = {"autoload": True}


`__mapper_args__ = {'column_prefix':'bb_'}` will prefix all column keys with '_bb'. Now, we have got the tables set up, reflected and mapped, we can get any query from tabels.

In [7]:
metadata = Base.metadata
Session = sqlalc.orm.sessionmaker()
Session.configure(bind=engine)
session = Session()

### Query for bb values

In [8]:
bb_query = "bb_1233"
topn = 10
sel = [samplesTable.bb_otu_id,
       samplesTable.bb_otu_label,
       getattr(samplesTable, bb_query)]
query = session.query(*sel) \
    .filter(getattr(samplesTable, bb_query)) \
    .order_by(sqlalc.desc(getattr(samplesTable, bb_query))) \
    .all()[0:topn]
d = list(map(lambda x: {"otu_id": x[0],
                        "otu_label": x[1],
                        "value": x[2]},
             query))

## Get bb names
Access table data through `__table__` attribute:

In [9]:
bb_names = samplesTable.__table__.columns.keys()[2:]
bb_names

['940',
 '941',
 '943',
 '944',
 '945',
 '946',
 '947',
 '948',
 '949',
 '950',
 '952',
 '953',
 '954',
 '955',
 '956',
 '958',
 '959',
 '960',
 '961',
 '962',
 '963',
 '964',
 '966',
 '967',
 '968',
 '969',
 '970',
 '971',
 '972',
 '973',
 '974',
 '975',
 '978',
 '1233',
 '1234',
 '1235',
 '1236',
 '1237',
 '1238',
 '1242',
 '1243',
 '1246',
 '1253',
 '1254',
 '1258',
 '1259',
 '1260',
 '1264',
 '1265',
 '1273',
 '1275',
 '1276',
 '1277',
 '1278',
 '1279',
 '1280',
 '1281',
 '1282',
 '1283',
 '1284',
 '1285',
 '1286',
 '1287',
 '1288',
 '1289',
 '1290',
 '1291',
 '1292',
 '1293',
 '1294',
 '1295',
 '1296',
 '1297',
 '1298',
 '1308',
 '1309',
 '1310',
 '1374',
 '1415',
 '1439',
 '1441',
 '1443',
 '1486',
 '1487',
 '1489',
 '1490',
 '1491',
 '1494',
 '1495',
 '1497',
 '1499',
 '1500',
 '1501',
 '1502',
 '1503',
 '1504',
 '1505',
 '1506',
 '1507',
 '1508',
 '1510',
 '1511',
 '1512',
 '1513',
 '1514',
 '1515',
 '1516',
 '1517',
 '1518',
 '1519',
 '1521',
 '1524',
 '1526',
 '1527',
 '1530'

## Query metadara

In [17]:
bb_query = "bb_940"
bb_query_number = int(bb_query.split("_")[1])
sel = [samplesMetaDataTable.BBTYPE,
       samplesMetaDataTable.AGE,
       samplesMetaDataTable.GENDER,
       samplesMetaDataTable.CAT,
       samplesMetaDataTable.ETHNICITY,
       samplesMetaDataTable.LOCATION,
       samplesMetaDataTable.COUNTRY012,
       samplesMetaDataTable.MMAXTEMP013,
       samplesMetaDataTable.sample]
query = session.query(*sel).filter(samplesMetaDataTable.sample == bb_query_number).first()
d = {"type": query[0],
     "age": query[1],
     "gender": query[2],
     "category": query[3],
     "ethnicity": query[4],
     "location": query[5],
     "country": query[6],
     "temperature": query[7]}
d

{'type': 'I',
 'age': 24.0,
 'gender': 'F',
 'category': 'no',
 'ethnicity': 'Caucasian',
 'location': 'Beaufort/NC',
 'country': 'usa',
 'temperature': 54.5}

In [11]:
session.query(samplesMetaDataTable.sample, samplesMetaDataTable.ZIP012).all()

[(940, '22306'),
 (941, None),
 (943, None),
 (944, '7079'),
 (945, '84404'),
 (946, '12538'),
 (947, '48823'),
 (948, '27613'),
 (949, '7009'),
 (950, '1730'),
 (952, None),
 (953, None),
 (954, '45429'),
 (955, None),
 (956, None),
 (958, None),
 (959, '93605'),
 (960, None),
 (961, 'K7V3Z4'),
 (962, '60514'),
 (963, '45238'),
 (964, None),
 (966, '7924'),
 (967, None),
 (968, None),
 (969, '7071'),
 (970, None),
 (971, '10533'),
 (972, '55987'),
 (973, '28269'),
 (974, '14905'),
 (975, '84404'),
 (978, None),
 (1233, '97374'),
 (1234, '13209'),
 (1235, '12603'),
 (1236, '28054'),
 (1237, '27517'),
 (1238, '27834'),
 (1242, None),
 (1243, None),
 (1246, '30076'),
 (1253, None),
 (1254, None),
 (1258, None),
 (1259, None),
 (1260, None),
 (1264, '1066'),
 (1265, '27615'),
 (1273, '98022'),
 (1275, None),
 (1276, None),
 (1277, '35223'),
 (1278, None),
 (1279, None),
 (1280, '85706'),
 (1281, '63348'),
 (1282, None),
 (1283, '27608'),
 (1284, '27612'),
 (1285, None),
 (1286, '43420'),
